# Galaxy Spectra

Unlike component spectra, galaxy level spectra are by definition integrated. These spectra are combinations of different component spectra. To see how to define emission models for galaxy level spectra see the [combined emission](../emission_models/combined_models.ipynb) docs. Here we will demonstrate the usage of a combined model to generate spectra for a galaxy.

## Integrating particle spectra

## Getting observed spectra